In [3]:
import json
import re

In [14]:
questions = dict()
cal_reasoning = 0
with open('../data/gpt-4o-mini__v3.jsonl') as f:
    for line in f:
        data = json.loads(line)
        ids = data['ids']
        questions[ids] = data
        if 'reasoning' in data:
            cal_reasoning += 1
cal_reasoning

1765

In [15]:
len(questions)

1765

In [16]:
with open('../data/meta-llama__Llama-3.3-70B-Instruct-Turbo-scored-gpt-4o-mini__v3.jsonl') as f:
    for line in f:
        data = json.loads(line)
        ids = data['ids']
        if 'score' not in questions[ids]:
            questions[ids]['score'] = []
        questions[ids]['score'].append(data['score'])

with open('../data/gemini-1.5-flash-scored-gpt-4o-mini__v3.jsonl') as f:
    for line in f:
        data = json.loads(line)
        ids = data['ids']
        if 'score' not in questions[ids]:
            questions[ids]['score'] = []
        questions[ids]['score'].append(data['score'])

with open('../data/gpt-4o-mini-scored-gpt-4o-mini__v3.jsonl') as f:
    for line in f:
        data = json.loads(line)
        ids = data['ids']
        if 'score' not in questions[ids]:
            questions[ids]['score'] = []
        questions[ids]['score'].append(data['score'])

In [17]:
len_sql = 0
for ids, data in questions.items():
    if 'score' in data:
        data['avg'] = sum(data['score']) / 3 # len(data['score'])
    len_sql += len(data['sql'])
len_sql /= len(questions)
len_sql


0.9949008498583569

In [18]:
good_ids = set()
for ids, data in questions.items():
    if 'score' in data and data['avg'] == 1:
        good_ids.add(ids)
        if len(data['sql']) > 1:
            print(ids, data)

In [19]:
len(good_ids)

879

In [20]:
with open('../data/gpt-4o-mini__v3_good.jsonl', 'w') as f:
    for ids, data in questions.items():
        if ids in good_ids:
            f.write(json.dumps(data) + '\n')

In [13]:
count_result_format = 0
good_result_format = 0
for ids, data in questions.items():
    if 'result format' in data['reasoning'].lower():
        count_result_format += 1
        if '### Result Format' in data['reasoning']:
            good_result_format += 1
            print(ids, data['reasoning'])
            break
count_result_format, good_result_format

(0, 0)

In [10]:
for ids, data in questions.items():
  if ids == '74710a9c-1b03-4821-b306-61057c8200cf':
    print('QUESTION ===========')
    print(data['question'])  
    print('TABLES ===========')
    print(data['table'])
    print('ANSWERS ===========')
    print(data['reasoning'])
    print('AVG ===========')
    print(data['avg'])
    break

QUESTION ===========
In Q2 2024, analyze the accounts receivable turnover and days sales outstanding (DSO) for FPT, Vingroup (VIC), and Masan Group (MSN). How do these metrics reflect their collection efficiency, and how might their subsidiaries/investments affect these figures?
TABLES ===========
**Company Info**

| stock_code   | company_name                   | en_company_name              | industry               |
|:-------------|:-------------------------------|:-----------------------------|:-----------------------|
| FPT          | Công ty Cổ phần FPT            | FPT Corporation              | Information Technology |
| MSN          | Công ty Cổ phần Tập đoàn Masan | Masan Group Corporation      | Food and Beverages     |
| VIC          | Tập đoàn Vingroup - Công ty CP | VinGroup Joint Stock Company | Real Estate            |

**category_code_ratio**

| ratio_code   | ratio_name                 |
|:-------------|:---------------------------|
| RTSR         | Receivables To Sal

In [21]:
import random


In [22]:
random.randint(0, 1)

1

In [6]:
import tiktoken

def count_tokens(text: str, model: str = "gpt-4") -> int:
    # Select the tokenizer for the model
    encoding = tiktoken.encoding_for_model(model)
    tokens = encoding.encode(text)
    return len(tokens)

err = 0
total_tokens = 0
num = 1
s = 0
u = 0
a = 0
with open('../data/message_gpt-4o-mini__v3_good.jsonl') as f:
    for line in f:
        try:
            data = json.loads(line)

            with open('../data/message_gpt-4o-mini__v3_good_good.jsonl', 'a') as f:
                json.dump(data, f)
                f.write('\n')

            for message in data:
                text = message['content']
                total_tokens += count_tokens(text)
                if message['role'] == 'system':
                    s += count_tokens(text)
                elif message['role'] == 'user':
                    u += count_tokens(text)
                else:
                    a += count_tokens(text)
        except Exception as e:
            err += 1
        num += 1
total_tokens, err, u, s, a

(2480381, 16, 955872, 1405070, 119439)

In [2]:
def is_npl(question: dict):
    if 'non-per' in question['question'].lower() and 'bdr' not in question['reasoning'].lower():
        return True
    return False

In [7]:
file_path = '../data/gpt-4o-mini__v3.jsonl'
new_file_path = '../data/gpt-4o-mini__v3_2.jsonl'

In [8]:
with open(file_path) as f:
    for line in f:
        data = json.loads(line)
        if not is_npl(data):
            with open(new_file_path, 'a') as f:
                json.dump(data, f)
                f.write('\n')